In [ ]:
import httpx
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

tqdm.pandas()

In [ ]:
xls = pd.ExcelFile('../../data/stories.xlsx', engine='openpyxl')
df = pd.read_excel(xls, 'Articles')

In [ ]:
get_id = lambda x: Path(x).name

In [ ]:
df['id'] = df['URL'].apply(get_id)

In [ ]:
a = f'[at(document.id, "{id}")]'

In [ ]:
def get_story_text(id):
    response = httpx.get(
        "https://wellcomecollection.cdn.prismic.io/api/v2/documents/search",
        params={
            "ref": "YZZ4mREAACcAWBme",
            "q": f'[[at(document.id, "{id}")]]'
        }
    ).json()

    slices = response['results'][0]['data']['body']
    paragraphs = [
        paragraph['text']
        for slice in slices
        if slice['slice_type'] == 'text'
        for paragraph in slice['primary']['text']
    ]
    text = '\n'.join(paragraphs)
    return text

In [ ]:
df['fulltext'] = df['id'].progress_apply(get_story_text)

In [ ]:
df.to_json('../../data/stories.json', orient='records')